In [ ]:
import dapla as dp
import pandas as pd

In [ ]:
#dp.show('/felles/mock_sysselsatte')

In [ ]:
dp.show_meta('/felles/mock_sysselsatte/companies_2019_30000')

In [ ]:
comp = dp.read_pandas('/felles/mock_sysselsatte/companies_2019_30000')
pop = dp.read_pandas('/felles/mock_sysselsatte/population_2019_30000')

In [ ]:
%%output


In [ ]:
pop

In [ ]:
# Aldersfordeling i populasjonen
pop['age'].value_counts().sort_index().plot.line()

In [ ]:
comp

In [ ]:
# Kan også hente kolonner med punktnotasjon, men ligner da mer på andre ting...
comp.nace
# Men det er strengt tatt færre tegn

### Sletting

In [ ]:
# Droppe kolonner
comp.drop('employee_points', axis = 1)

In [ ]:
#comp

In [ ]:
# Finne duplikater på nace
comp[comp.duplicated('nace', keep = False)].sort_values('nace')

In [ ]:
# Droppe duplikater ved å bruke .drop_duplicates
comp.drop_duplicates('nace').sort_values('nace')

In [ ]:
# Droppe duplikater ved å gruppere, og beholde den første
comp.groupby('nace', as_index = False).first()
# OBS, du mister indexen på raden!!!

### Ofte kan det være smart å filtrere ned først
Men lag en kopi av orginaldatasettet før du gjør permanente endringer

In [ ]:
comp_tmp = comp

In [ ]:
comp_tmp

In [ ]:
# Enkelt filter Oslo
comp_tmp_oslo = comp_tmp[comp_tmp['region_code'] == '03']
print(len(comp_tmp_oslo))
comp_tmp_oslo

In [ ]:
# Invert filter
comp_tmp_ikkeoslo = comp_tmp[comp_tmp['region_code'] != '03']
comp_tmp_ikkeoslo = comp_tmp[~(comp_tmp['region_code'] == '03')]
comp_tmp_ikkeoslo

In [ ]:
# Enkelt filter Oslo eller Østfold, "|" = "or"
comp_tmp_osloostfold = comp_tmp[(comp_tmp['region_code'] == '03') | (comp_tmp['region_code'] == '01')]
comp_tmp_osloostfold

In [ ]:
# Isin Oslo eller Østfold
comp_tmp_osloostfold = comp_tmp[comp_tmp['region_code'].isin(['03', '01'])]
comp_tmp_osloostfold

In [ ]:
# unique
unike_regionkoder = pd.unique(comp['region_code']).tolist()
unike_regionkoder

In [ ]:
# Skriver sort over listen? ja.
unike_regionkoder[:-1].sort()
# The sort() method doesn't return any value. Rather, it changes the original list.

In [ ]:
unike_regionkoder[:5]

In [ ]:
comp[comp['region_code'].isin(unike_regionkoder[:5])]

### "Kobling" (merge + concat)

In [ ]:
# Merge (alle varibaler)
pop.merge(comp, how = 'left')
#pop.merge(comp, how = 'left', left_on = 'work_id', right_on='work_id')

In [ ]:
pop_2020 = dp.read_pandas('/felles/mock_sysselsatte/population_2020_30000')

# Concat, slå sammen lignende ting etter hverandre (ogsp kalt append)
fnr_20192020 = pd.concat([pop_2020['id'], pop['id']])
fnr_20192020

In [ ]:
len(pd.unique(fnr_20192020))

In [ ]:
comp_tmp = comp

In [ ]:
comp_tmp[comp_tmp['nace'] == '01.000u']

In [ ]:
# .loc for å sette verdier i en annen kolonne
comp_tmp.loc[comp_tmp['nace'] == '01.000u', 'employee_points'] = '0'
# Se på de som evt. ble omkodet
comp_tmp[comp_tmp['nace'] == '01.000u']

In [ ]:
# loop gjennom pandas-katalog med iterrows()
for i, row in comp.iterrows():
    wrk_id = row['work_id']
    print(wrk_id, row['nace'])
    #print(len(pop[pop['work_id'] == wrk_id]))

In [ ]:
# Random sample av rader, på størrelse 10
demo_sample = comp.sample(10)
demo_sample

In [ ]:
# .iloc vs. .loc

# iloc opererer uavhengig av indexen, den plukker bare "de 4 første", selv om de radene heter "2, 5, 6, 8"
demo_sample.iloc[0]

In [ ]:
# Mens loc går mer på index-navn, så her må raden hete "10", ikke være nummer 10
comp.loc[10]

### Påfør radene originaldatasettet

In [ ]:
# Bevissthet rundt radnummer / index

In [ ]:
# Doble alle indeksene, så vi ser forskjell på de, og de som kommer fra reset_index lengre ned
comp_tmp.index *= 2 

In [ ]:
# Ta en kopi av dataframen, for å være sikker på at vi ikke endrer det orginale datasettet 
comp_tmp2 = comp_tmp.copy()
# Egentlig så er pandas kanskje smart nok til å skjønne dette...

In [ ]:
# .reset_index() er litt skummel
comp_tmp2.head(5)

In [ ]:
comp_tmp2 = comp_tmp2.reset_index()
comp_tmp2.head(5)
# yay, dobbel index! Vil vi egentlig ha det?

In [ ]:
comp_tmp2 = comp_tmp.copy()
comp_tmp2 = comp_tmp2.reset_index(drop = True)
# Hva skjer om vi dropper på reset_index() ?
comp_tmp2

### Ryddighet

In [ ]:
# del for å være ryddig 

# Sletter 
del comp_tmp

In [ ]:
# valuation og state på dp.write_pandas()

dp.write_pandas(comp, '/felles/mock_sysselsatte/companies_2019_30000test', valuation = 'OPEN', state = 'TEMP')

# skriv dataframen tilbake til daplalagringen med en del options og path.
# Verdivurderingsscorenivå:    .option("valuation", "XXXX") = SENSITVE, SHIELDED, INTERNAL eller OPEN
# Dataferdigstillingssnivå:    .option("state", "XXXX") = RAW, INPUT, PROCESSED, OUTPUT, PRODUCT, TEMP eller OTHER
# Se kodekilde: https://github.com/statisticsnorway/dapla-catalog-protobuf/blob/master/src/main/proto/dataset.proto